there are some problems with huggingsound dependencies, I managed to get it working with:

```
pip install huggingsound 
pip install transformers==4.25
```

In [ ]:
from pathlib import Path
# this is using original huggingsound, not modified version from project files
from huggingsound import SpeechRecognitionModel
from huggingsound.speech_recognition.model import TrainingArguments
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

np.random.seed(33)

In [ ]:
preprocessed_data_dir = Path('./speech_finetuning_data')
with open(preprocessed_data_dir.joinpath('transcriptions.txt')) as f:
    lines = f.readlines()

dataset = []
for line in lines:
    sep_idx = line.index(':')
    filename = line[:sep_idx]
    transcription = line[sep_idx + 2:-1]
    file_path = preprocessed_data_dir.joinpath(filename)
    assert file_path.exists()
    dataset.append({
        'path': str(file_path.absolute()),
        'transcription': transcription
    })

In [ ]:
train, test = train_test_split(dataset, test_size=0.15)
train, eval = train_test_split(train, test_size=0.15)

In [ ]:
with open('./speech_finetuning_data/test_data_split.json', 'wb') as f:
    pickle.dump(test, f)

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-polish", device='cuda')
output_dir = './speech_model_finetuned'
model.finetune(
    output_dir,
    train_data=train,
    eval_data=eval,
    training_args=TrainingArguments(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        learning_rate=2e-4,
        num_train_epochs=2,
        eval_steps=40,
        logging_steps=40,
        gradient_accumulation_steps=1,
    )
)

In [ ]:
model.evaluate(test, decoder=None)